In [1]:
import pandas as pd
import json
import os
import requests
import base64
import time
import numpy as np
from datetime import datetime

IS_RASPI = False

if IS_RASPI:
    import RPi.GPIO as io     
    io.setmode(io.BCM)
    io.setup(14, io.OUT)
    time.sleep(1)
    io.output(14,0)
    time.sleep(1)
    io.output(14,1)

CONFIG_FILE = "config.json"

In [6]:
def encode_client(client_id = "", client_secret = "", **kwargs):
    return str(base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')),'utf-8')

def getToken(client_id = None, client_secret = None, **kwargs):
    assert client_id and client_secret, "client_id o client_secret están vacíos"
    os.system(f"curl --request POST --url 'https://www.moneybutton.com/oauth/v1/token' --header 'content-type: application/x-www-form-urlencoded' --header 'Authorization: Basic {encode_client(client_id, client_secret)}' --data grant_type=client_credentials   --data scope=application_access:write > token.txt ")
    token = json.loads(open("token.txt").read())
    return token

def saveConfig(client_id = "", client_secret = "", expires_in = 0, limit = 100, offset = 0, **kwargs):
    config = {"client_id" : client_id, "client_secret" : client_secret, "expire":(time.time() + 3600 ) if expires_in else 0, "limit": limit, "offset": offset}
    json.dump({**config,**kwargs},fp=open(CONFIG_FILE,"w+"))
    if not client_id or not client_secret:
        msg = f"""You need to complete your {CONFIG_FILE}

If you still don't have an app, go to:
https://www.moneybutton.com/settings/apps/create
and create one.
"""
        print(msg)

def readConfig():
    config = {}
    if os.path.isfile(CONFIG_FILE):
        config = json.loads(open(CONFIG_FILE).read())
    else:
        saveConfig()
        return None
    
    if config["client_id"] and config["client_secret"]:
        if not config["access_token"] or not config["expire"] or float(config["expire"]) < time.time():
            newToken = getToken(**config)
            newToken["access_token"]
            config["expire"] = time.time()+newToken["expires_in"]
            config = {**config, **newToken}
            saveConfig(**config)
        return config
    
    return config

def decodeBytestring(bytestring):
    return bytes.fromhex(bytestring).decode("utf8")

def getTransactionScripts(access_token, limit = 10, offset = 0, **kwargs):
    headers = {'Authorization': 'Bearer ' + access_token}
    response = requests.get("https://www.moneybutton.com/api/v1/payments", headers=headers, params={"scope":"application_access:write", "limit":limit, "offset":offset})
    if response.status_code == 200:
        response = response.json()
        return response
    else:
        print("Status Code:", response.status_code)
        return None

def getCommands(json):
    scripts = list(filter(lambda x: x["attributes"]["script"] and x["attributes"]["script"].startswith('0 OP_RETURN 6d6f6e6579627574746f6e2e636f6d 75746638 '), json["included"]))
    attributes = list(map(lambda x: x["attributes"], scripts))
    return attributes
    
def execute(turn, element_id, timer=None):
    #assert element_id == "1", "Solo tenemos id=1"
    turn = turn.upper()
    if turn == "OFF": # entiendo que es off
        print("Apagando", element_id)
        io.output(14,1) if IS_RASPI else None
    elif turn == "ON":
        print("Encendiendo", element_id)
        io.output(14,0) if IS_RASPI else None
    else:
        print("Comando desconocido: ", turn)

def updateLimits(config):
    config = {**config,**{"limit":100}}
    data = getTransactionScripts(**config)
    increment = len(data["data"])
    config["offset"] += (increment-4) if increment >=4 else 0
    config["limit"] = 20
    print("Offset set to: ",config["offset"])
    saveConfig(**config)
    

In [7]:
config = readConfig()
updateLimits(config)

Offset set to:  90


In [8]:
df = pd.DataFrame.from_records(getCommands(getTransactionScripts(**config)))
df.sort_values(by=["created-at"], ascending=False, inplace=True)
df["text"] = df.script.apply(lambda x: decodeBytestring(x.split(" ",4)[4]))
df

,created-at,to,amount,currency,satoshis,type,user-id,address,script,amount-usd,user-paymail,text
1,2019-09-09T15:45:58.573Z,None,0,BSV,0,SCRIPT,None,None,0 OP_RETURN 6d6f6e6579627574746f6e2e636f6d 757...,0,None,#OFF#1
3,2019-09-09T15:45:49.468Z,None,0,BSV,0,SCRIPT,None,None,0 OP_RETURN 6d6f6e6579627574746f6e2e636f6d 757...,0,None,#ON#1
0,2019-09-09T15:41:41.980Z,None,0,BSV,0,SCRIPT,None,None,0 OP_RETURN 6d6f6e6579627574746f6e2e636f6d 757...,0,None,#OFF#1
2,2019-09-09T15:41:22.904Z,None,0,BSV,0,SCRIPT,None,None,0 OP_RETURN 6d6f6e6579627574746f6e2e636f6d 757...,0,None,#ON#1


In [9]:
_, turn, element_id = dict(df.iloc[0])["text"].split("#", 3)

execute(turn,element_id)

Apagando 1
